In [ ]:
# needed: Python 3.10-3.11

# requirements:
## pm4py>=2.7.11.13
## duckdb
## openai
## crewai
## crewai[tools]
## langchain
## langchain-openai

# you can install them with the command: pip install -U pm4py>=2.7.11.13 duckdb openai crewai crewai[tools] langchain langchain-openai

# in Colab, you can uncomment the following line:
# !pip install pm4py>=2.7.11.13 duckdb openai crewai crewai[tools] langchain langchain-openai

In [ ]:
# imports the necessary requirements
from crewai import Crew, Agent, Task
from langchain_openai import ChatOpenAI
import pm4py
import duckdb
import os
from crewai_tools import tool
import pickle

In [ ]:
# sets the API key (always needed!)
os.environ["OPENAI_API_KEY"] = open("api_key.txt", "r").read()

# sets the address of the APIs and the required model

# DeepInfra Qwen2-7B-Instruct
llm = ChatOpenAI(model="Qwen/Qwen2-7B-Instruct", base_url="https://api.deepinfra.com/v1/openai/")

# OpenAI's GPT-4o
#llm = ChatOpenAI(model="gpt-4o", base_url="https://api.openai.com/v1")

# OpenAI's GPT-3.5-turbo
#llm = ChatOpenAI(model="gpt-3.5-turbo", base_url="https://api.openai.com/v1")

In [ ]:
# reads the event log in-memory using pm4py
log = pm4py.read_xes("renting_log_high.xes.gz")

# computes the DFG abstraction
dfg_abstraction = pm4py.llm.abstract_dfg(log)

In [ ]:
# defines three agents:
# - the first one specialized in root cause analysis identification
# - the second one specialized in explaining the root cause analysis insights
# - the third one acting as a judge, giving a score from 1.0 to 10.0 to the previous insights
# the agents have all a 'role', a 'goal', a 'backstory', and are connected to the provided LLM.

pm_root_cause_analyst = Agent(role="root_cause_analyst", goal="Perform root cause analysis in process mining.", backstory="Finds all the root causes of the performance issues.", llm=llm)
pm_root_cause_explainer = Agent(role="root_cause_explainer", goal="Explaining process mining insights to non-technical people.", backstory="Very detailed analyst.", llm=llm)
pm_insights_grader = Agent(role="insights_grader", goal="Giving a score to the process mining insights proposed by others.", backstory="Judge.", llm=llm)


In [ ]:
# defines the tasks of the three agents

root_cause_analysis = Task(description="Find the root causes of the performance issues in the process. Only process and data specific considerations, not general considerations. DFG abstraction: {dfg_abstraction}", expected_output="The chain of thought related to the insight", agent=pm_root_cause_analyst)
root_cause_chain_of_thought = Task(description="Explain the first of the provided insight in detail. Original DFG abstraction: {dfg_abstraction}", expected_output="The chain of thought related to the insight", agent=pm_root_cause_explainer)
root_cause_grader = Task(description="Provide a grade from 1.0 (minimum) to 10.0 (maximum) for all the provided root cause insights. Original DFG abstraction: {dfg_abstraction}", expected_output="A list of insights, each accompanied by a grade.", agent=pm_insights_grader)


In [ ]:
# defines the crew

print(os.environ["OPENAI_API_KEY"])
crew = Crew(agents=[pm_root_cause_analyst, pm_root_cause_explainer, pm_insights_grader], tasks=[root_cause_analysis, root_cause_chain_of_thought, root_cause_grader])

In [ ]:
# starts the crew

crew.kickoff({"dfg_abstraction": dfg_abstraction})